In [2]:
import iris
import glob
import netCDF4
import numpy as np
import os
import xarray as xr

In [3]:
files = glob.glob('/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/*nc')
print(files)

file = files[0]
print(file)

['/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_175001-179912.nc', '/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_180001-184912.nc', '/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_185001-185012.nc', '/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_185101-189912.nc', '/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_190001-194912.nc', '/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_195001-199912.nc', '/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/CO2-em-anthro_input4

In [4]:
def global_monthly_sum(in_cube):
    # Global area weighted TOTAL CO2 emissions
    cube = in_cube.copy()
    if not cube.coord('latitude').has_bounds():
        cube.coord('latitude').guess_bounds()
    if not cube.coord('longitude').has_bounds():
        cube.coord('longitude').guess_bounds()
    grid_areas = iris.analysis.cartography.area_weights(cube)
    cube = cube.collapsed(['longitude', 'latitude', 'sector'], iris.analysis.SUM, weights=grid_areas)
    return cube, grid_areas

def comp_area_lat_lon(latvals,lonvals):
    """
    Code adapted from Ana Bastos -Max Planck Institue for Biogeochemistry, Jena
    This function will calculate the area of the gridpoints using the 
    latitude and longitudes of the dataset
    input: latvals - a 1-D array of the latitudes 
         : lonvals - a 1-D array of the longitudes
    
    return: area - the area (in metres-squared) of the gridcells"""
    #the radius of the Earth
    radius = 6.37122e6 # in meters
  
    #remove single-dimensional entires from the shape of an array
    lat=np.squeeze(latvals); lon=np.squeeze(lonvals)
    #get the length of the long itude and latitude
    nlat=len(lat)
    nlon=len(lon)
    
    # LATITUDE
    lat_edge=np.zeros((nlat+1))
    lat_edge[0] = max(-90, lat[0]-0.5*(lat[1]-lat[0])); 
    lat_edge[1:nlat] = 0.5*(lat[0:nlat-1] + lat[1:nlat])
    lat_edge[nlat] = min(90, lat[nlat-1]-0.5*(lat[nlat-2]-lat[nlat-1]))
    #calculate the n-th discrete difference
    dlat=np.diff(lat_edge)
    
    #LONGITUDE
    lon_edge=np.zeros((nlon+1))
    lon_edge[0] = lon[0]-0.5*(lon[1]-lon[0])
    lon_edge[1:nlon] = 0.5*(lon[0:nlon-1] + lon[1:nlon])
    lon_edge[nlon] = lon[nlon-1]-0.5*(lon[nlon-2]-lon[nlon-1])
    dlon=np.diff(lon_edge)
    
    dlon_2d, dlat_2d = np.meshgrid(dlon,dlat) # create mesh with cell size in deg    
    lon_2d, lat_2d = np.meshgrid(lon, lat)

    #convert latitudes to radians
    dy = radius * (dlat_2d * (np.pi/180.0)) 
    dx = radius * np.multiply(dlon_2d * (np.pi/180.0),np.cos(lat_2d * (np.pi/180.0)))

    area = np.multiply(dx , dy)
    if np.sum(area)<0:
        area=-1*area

    return area


In [ ]:
f1 = global_monthly_sum(iris.load_cube(file))

/opt/jaspy/lib/python3.7/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [ ]:
f1 = netCDF4.Dataset(file)
lon = f1.variables["lon"]
lonvals = lon[:]
lat = f1.variables["lat"]
latvals = lat[:]
time = f1.variables["time"][:]

carbon = f1.variables["CO2_em_anthro"]
print(carbon.dimensions)

In [ ]:
grid_area=comp_area_lat_lon(latvals,lonvals)

In [ ]:
grid_area

In [ ]:
dataset = xr.open_mfdataset('/gws/pw/j05/cop26_hackathons/bristol/project08/co2_emiss/raw_files_hist/*nc')

In [ ]:
carbon = dataset["CO2_em_anthro"]

In [ ]:
all_carbon = carbon.sum(dim="sector")

In [ ]:
all_carbon.sizes

In [ ]:
lon = all_carbon["lon"][:]
print(len(lon))
lat = all_carbon["lat"][:]
print(len(lat))
grid_area=comp_area_lat_lon(latvals,lonvals)
grid_area.shape

In [ ]:
all_carbon["time"]=xr.CFTimeIndex.to_datetimeindex(all_carbon["time"])
all_carbon.sizes

In [ ]:
all_carbon2 = all_carbon*grid_area
all_carbon2.sizes
all_carbon2

In [ ]:
all_carbon3 = all_carbon2.sum(dim=["lat", "lon"])

In [ ]:
all_carbon3.sizes

In [ ]:
all_carbon4 = all_carbon3.resample(time="1Y").mean() # we take the mean here because in the next line we are converting seconds > years

#convert kg to GtCO2

all_carbon5 = all_carbon4*60*60*24*365*1e-12 # *(44/12)> the ipcc figure is in GtC not CO2.
all_carbon5.to_pandas().to_csv("annual_carbon_V2.csv")
